In [1]:
import gymnasium as gym
from stable_baselines3 import PPO,DQN
from config import config
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import BaseCallback

env_id = "intersection-v0"
num_cpu = 4
env = make_vec_env(env_id, n_envs=num_cpu,env_kwargs={"config":config})
log_dir = "logs"

c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment intersection-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [3]:
ppo_params = { # paramètres de base choisis
    "n_steps": 64,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 64,
    'tensorboard_log':log_dir
}

ppo_optuna = { # meilleurs paramètres trouvés par optuna
    "n_steps": 1298,
    "gamma": 0.98,
    "learning_rate": 0.0001291858786170927,
    "ent_coef": 0.0016824274086785702,
    "clip_range": 0.19565974040447376,
    "n_epochs": 1,
    'tensorboard_log':log_dir
    }

ppo_optuna_modif = { #modifiation des paramètres optuna ( batch_size = 128, n_epochs = 3)
    "n_steps": 1024,
    "gamma": 0.98,
    "learning_rate": 0.0001291858786170927,
    "ent_coef": 0.0016824274086785702,
    "clip_range": 0.19565974040447376,
    "n_epochs": 3,
    "batch_size": 128,
    'tensorboard_log':log_dir
    }

model = PPO('CnnPolicy',env, verbose=1,**ppo_optuna_modif)
# model = PPO.load("test_PPO_intersection_matthias5")

Using cpu device


c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
# Création et évaluation d'un agent sur une policy non-entraîné

eval_env = gym.make(env_id, render_mode="rgb_array",config=config)
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward:.2f}")

c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment intersection-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(
c:\Users\matth\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward: 16.0 +/- 6.93


In [5]:
class DoneCallback(BaseCallback):
    """Save model every save_freq calls and print done count every check_freq calls
    """
    def __init__(self, check_freq: int, save_freq: int, save_path: str, verbose=1):
        super(DoneCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_freq = save_freq 
        self.save_path = save_path
        self.done_count = 0

    def _on_step(self) -> bool:
        if 'done' in self.locals.keys():
            if self.locals['done']:
                self.done_count += 1

        if self.n_calls % self.check_freq == 0:
            print(f"Step: {self.num_timesteps} Done count: {self.done_count}")

        if self.n_calls % self.save_freq == 0:
            self.model.save(self.save_path + str(self.num_timesteps))

        return True

In [ ]:
callback = DoneCallback(check_freq=128, save_freq=2500, save_path="./models/dqn_cnn")
# model.set_env(env)
model.learn(total_timesteps=512*1000,callback=callback, progress_bar=True)
model.save("test_PPO_intersection_matthias5_5")

In [9]:
# Evaluate the trained agent
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward:.2f}")

Mean reward: 11.4 +/- 8.69


In [10]:
%matplotlib inline
import matplotlib.pyplot as plt

# Visualize the trained agent in the simulation
while True:
  done = truncated = False
  obs, info = eval_env.reset()
  while not (done or truncated):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = eval_env.step(action)
    eval_env.render()



AttributeError: 'NoneType' object has no attribute 'get_image'